In [5]:
from ktms import *
from MAInit import *
from MACalc import *
from MAPost import *
import re

In [6]:
from pymongo import MongoClient
client = MongoClient()

In [7]:
client = MongoClient('localhost', 27017)

In [8]:
db = client.adsE_database
collection = db.adsE_collection

In [9]:
def makejson(file):
    atoms = query(file, 'spacom')

    dic = {}

    res = re.match('(.*u\d).*', file)
    barefile = res.group(1) + '.traj'
    bareatoms = query(barefile, 'spacom')
#     print(bareatoms)

    COatoms = query('CO.traj', 'scpacom')
    COene = COatoms.get_potential_energy()

    res = re.match('(\w.*)_(\d{3}).*', file)
    ele = res.group(1)
    mir = res.group(2)
    
    num = 0
    ads = None
    res = re.match('.*n(\d+)_', file)
    if res:
        num = int(res.group(1))
        
    res = re.match('.*u(\d+).*', file)
    unit = int(res.group(1))

    res = re.match('.*no.*_(\w.)_n', file)
    if res:
        ads = res.group(1)

    ### calc area ###
    x = atoms.cell[0][0]
    y = atoms.cell[1][1]
    area = x*y

    ### calc adsorvates number ###
    tot = 0
    for atom in atoms:
        if atom.tag == 1:
            tot += 1

    ### get energies ###
    ene = atoms.get_potential_energy()
    bareene = bareatoms.get_potential_energy()
    totaladsene = ene - (COene*num + bareene)
    
    ### get adsorbate position info ###
    igroups, iposlis, rgroups, rposlis = getadsposinfo(file, 0)
    
    ### get COint info ###
    try:
        file_ = file[:-5] + '__.traj'
        atoms_ = query(file_, 'spacom')
        ene_ = atoms_.get_potential_energy()
    except:
        ene_ = 0
        
    ### get COlength info ###
    COlengthlis = []
    for i in range(len(atoms)):
        if atoms[i].symbol == 'C':
            Cpos = atoms[i].position
            Opos = atoms[i+1].position
            COlengthlis.append(np.linalg.norm(Cpos-Opos))
            
    dic['name'] = file
    dic['element'] = ele
    dic['mirror'] = mir
    dic['adsorbate'] = ads
    dic['unitlength'] = unit
    dic['surfatomnum'] = tot
    dic['area'] = area
    dic['Energy'] = ene
    dic['totaladsEnergy'] = totaladsene
    dic['aveadsEne/suratom'] = totaladsene/tot
    dic['numberofads'] = num
    dic['coverage'] = num/tot
    dic['density'] = num/area
    dic['igroups'] = igroups
    dic['iposlis'] = [list(item) for item in iposlis]
    dic['rgroups'] = rgroups
    dic['rposlis'] = [list(item) for item in rposlis]
    dic['ECO_Eint'] = ene_
    dic['Ediff'] = ene - ene_
    dic['COlengthlis'] = COlengthlis

    if num:
        dic['aveadsEne/ads'] = totaladsene/num
    else:
        dic['aveadsEne/ads'] = None
    maxdiff = getmaxdiff(file)

    if maxdiff > 1.5:
        dic['moved+'] = 'yes'
    else:
        dic['moved+'] = 'no'
    
    if igroups == rgroups:
        dic['moved'] = 'no'
    else:
        dic['moved'] = 'yes'
    
    for key in dic.keys():
        print('{}: {}'.format(key, dic[key]))
    
    return dic

In [10]:
env = 'local'
files = os.listdir('/home/katsuyut/database/')
for i in range(len(files)):
    if '.traj' in files[i]:
        if 'all' not in files[i]:
            if '__' not in files[i]:
                print(i, files[i])

1 Al_100_u1.traj
3 Al_100_u1_no001_CO_n1_d3.traj
6 Al_100_u1_no002_CO_n1_d3.traj
9 Al_100_u1_no003_CO_n1_d3.traj
12 Al_100_u2.traj
14 Al_100_u2_no001_CO_n1_d9.traj
17 Al_100_u2_no002_CO_n2_d3.traj
20 Al_100_u2_no003_CO_n3_d3.traj
23 Al_100_u2_no004_CO_n4_d3.traj
26 Al_100_u2_no005_CO_n3_d3.traj
29 Al_100_u2_no006_CO_n4_d3.traj
32 Al_100_u2_no007_CO_n2_d6.traj
35 Al_100_u2_no008_CO_n2_d4.traj
38 Al_100_u2_no009_CO_n3_d3.traj
41 Al_100_u2_no010_CO_n1_d9.traj
44 Al_100_u2_no011_CO_n2_d3.traj
47 Al_100_u2_no012_CO_n3_d3.traj
50 Al_100_u2_no013_CO_n4_d3.traj
53 Al_100_u2_no014_CO_n3_d3.traj
56 Al_100_u2_no015_CO_n4_d3.traj
59 Al_100_u2_no016_CO_n2_d3.traj
62 Al_100_u2_no017_CO_n2_d4.traj
65 Al_100_u2_no018_CO_n3_d3.traj
68 Al_100_u2_no019_CO_n2_d6.traj
71 Al_100_u2_no020_CO_n1_d9.traj
74 Al_100_u2_no021_CO_n2_d3.traj
77 Al_100_u2_no022_CO_n3_d3.traj
80 Al_100_u2_no023_CO_n4_d3.traj
83 Al_100_u2_no024_CO_n2_d6.traj
86 Al_100_u3.traj
88 Al_100_u3_no001_CO_n1_d15.traj
91 Al_100_u3_no002_CO_n1_

In [7]:
makejson(files[3])

Symmetrically same structure found!
name: Ag_100_u1_no01_CO_n1_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 1
surfatomnum: 1
area: 8.874959529172191
Energy: -22.44298044
totaladsEnergy: 0.1705699100000011
aveadsEne/suratom: 0.1705699100000011
numberofads: 1
coverage: 1.0
density: 0.11267657015369789
igroups: [0]
iposlis: [[0.0, 0.0, 18.169597920855004]]
rgroups: [0]
rposlis: [[5.521685380174929e-07, 5.944197952180715e-07, 18.562673073888003]]
ECO_Eint: -14.2275679
Ediff: -8.215412539999999
COlengthlis: [1.1576383705845288]
aveadsEne/ads: 0.1705699100000011
moved+: no
moved: no


{'name': 'Ag_100_u1_no01_CO_n1_d3.traj',
 'element': 'Ag',
 'mirror': '100',
 'adsorbate': 'CO',
 'unitlength': 1,
 'surfatomnum': 1,
 'area': 8.874959529172191,
 'Energy': -22.44298044,
 'totaladsEnergy': 0.1705699100000011,
 'aveadsEne/suratom': 0.1705699100000011,
 'numberofads': 1,
 'coverage': 1.0,
 'density': 0.11267657015369789,
 'igroups': [0],
 'iposlis': [[0.0, 0.0, 18.169597920855004]],
 'rgroups': [0],
 'rposlis': [[5.521685380174929e-07,
   5.944197952180715e-07,
   18.562673073888003]],
 'ECO_Eint': -14.2275679,
 'Ediff': -8.215412539999999,
 'COlengthlis': [1.1576383705845288],
 'aveadsEne/ads': 0.1705699100000011,
 'moved+': 'no',
 'moved': 'no'}

In [11]:
env = 'local'
files = os.listdir('/home/katsuyut/database/')

for i in range(len(files)):
    if '.traj' in files[i]:
        if 'all' not in files[i]:
            if 'CO.traj' not in files[i]:
                if '__' not in files[i]:
                    print(i, files[i])

                    post_data = makejson(files[i])
                    result = collection.insert_one(post_data)
                    print('One post: {0}\n'.format(result.inserted_id))

1 Al_100_u1.traj
Symmetrically same structure found!
name: Al_100_u1.traj
element: Al
mirror: 100
adsorbate: None
unitlength: 1
surfatomnum: 1
area: 8.243242894537874
Energy: -13.12909201
totaladsEnergy: 0.0
aveadsEne/suratom: 0.0
numberofads: 0
coverage: 0.0
density: 0.0
igroups: []
iposlis: []
rgroups: []
rposlis: []
ECO_Eint: 0
Ediff: -13.12909201
COlengthlis: []
aveadsEne/ads: None
moved+: no
moved: no
One post: 5d96450eb06310c5ac1cfe9a

3 Al_100_u1_no001_CO_n1_d3.traj
Symmetrically same structure found!
name: Al_100_u1_no001_CO_n1_d3.traj
element: Al
mirror: 100
adsorbate: CO
unitlength: 1
surfatomnum: 1
area: 8.243242894537874
Energy: -27.47946116
totaladsEnergy: 0.13506716000000196
aveadsEne/suratom: 0.13506716000000196
numberofads: 1
coverage: 1.0
density: 0.12131148054154982
igroups: [0]
iposlis: [[0.0, 0.0, 17.940533065785]]
rgroups: [0]
rposlis: [[-1.388840051976476e-08, 8.998811678742477e-09, 17.97991934719171]]
ECO_Eint: -14.11966132
Ediff: -13.359799839999999
COlengthlis:

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Al_100_u2_no005_CO_n3_d3.traj
element: Al
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 32.972971578151494
Energy: -96.09348715
totaladsEnergy: -0.22476847000000078
aveadsEne/suratom: -0.056192117500000194
numberofads: 3
coverage: 0.75
density: 0.09098361040616236
igroups: [1, 0, 0]
iposlis: [[1.4355524106191555, 2.8711048212383123, 17.940533065785], [2.871104821238311, 0.0, 17.940533065785], [0.0, 0.0, 17.940533065785]]
rgroups: [1, 0, 0]
rposlis: [[1.4355399436313219, 2.8711047415788085, 17.7576611324326

Symmetrically same structure found!
name: Al_100_u2_no011_CO_n2_d3.traj
element: Al
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 32.972971578151494
Energy: -81.72628345
totaladsEnergy: -0.3430010799999934
aveadsEne/suratom: -0.08575026999999835
numberofads: 2
coverage: 0.5
density: 0.06065574027077491
igroups: [1, 1]
iposlis: [[2.8711048212383123, 1.4355524106191555, 17.940533065785], [8.881784197001252e-16, 1.4355524106191555, 17.940533065785]]
rgroups: [1, 1]
rposlis: [[2.8711048030687243, 1.4355527371867642, 17.769543724576447], [9.064056100252706e-09, 1.4355522184399294, 17.769536548411768]]
ECO_Eint: -28.62238917
Ediff: -53.10389427999999
COlengthlis: [1.1908065240175707, 1.1907765895185878]
aveadsEne/ads: -0.1715005399999967
moved+: no
moved: no
One post: 5d96451eb06310c5ac1cfea9

47 Al_100_u2_no012_CO_n3_d3.traj
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetric

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Al_100_u2_no017_CO_n2_d4.traj
element: Al
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 32.972971578151494
Energy: -81.72511514
totaladsEnergy: -0.34183276999999634
aveadsEne/suratom: -0.08545819249999909
numberofads: 2
coverage: 0.5
density: 0.06065574027077491
igroups: [2, 1]
iposlis: [[1.4355524106191555, 4.306657231857467, 17.940533065785], [8.881784197001252e-16, 1.4355524106191555, 17.940533065785]]
rgroups: [2, 1]
rposlis: [[1.52879501478188, 4.306656854596593, 17.52050541633987], [-0.13112024192846458, 1.4355552093230355, 17.779957778568964]]
ECO_Eint: -28.98139788
Ediff: -52.74371726
COlengthlis: [1.1891703457971852, 1.2117384786827925]
aveadsEne/ads: -0.17091638499999817
moved+: no
moved: no
One post: 5d9645

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Al_100_u2_no024_CO_n2_d6.traj
element: Al
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 32.972971578151494
Energy: -80.77224748
totaladsEnergy: 0.6110348899999991
aveadsEne/suratom: 0.15275872249999978
numberofads: 2
coverage: 0.5
density: 0.06065574027077491
igroups: [2, 2]
iposlis: [[4.306657231857469, 4.306657231857469, 17.940533065785], [1.4355524106191555, 1.4355524106191555, 17.940533065785]]
rgroups: [2, 2]
rposlis: [[4.306657225492541, 4.306657233898371, 17.915082468732212], [1.4355524188275606, 1.

...

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Pd_111_u2_no022_CO_n3_d3.traj
element: Pd
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 27.42573780073108
Energy: -116.3503231
totaladsEnergy: -2.5548060799999917
aveadsEne/suratom: -0.6387015199999979
numberofads: 3
coverage: 0.75
density: 0.1093863006274358
igroups: [3, 3, 3]
iposlis: [[2.8137392471563696, 1.6245131117751455, 18.74222542477599], [4.220608870734555, 4.061282779437861, 18.74222542477599], [7.034348117890925, 4.061282779437861, 18.74222542477599]]
rgroups: [3, 3, 3]
rposlis: [[2.8138031133221437, 1.6414027617303562, 18.3699744745887], [4.20586162238504, 4.052778118882662, 18.370287993941556], [7.049014820181501, 4.0528891558036815, 18.37000769940702]]
ECO_Eint: -42.1

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure

In [12]:
for i in range(0,1):
    print(i)

0
